In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split, cross_val_score

import lightgbm as lgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import VotingRegressor

from sklearn import linear_model

In [2]:
train  = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')
sub = pd.read_excel('../MachineHackE-Commerce/Sample_Submission.xlsx')


In [3]:
train_embedding = pd.read_csv('../MachineHackE-Commerce/train_embedding.csv')
train_topic = pd.read_csv('../MachineHackE-Commerce/train_topic.csv')
train_tfidf = pd.read_csv('../MachineHackE-Commerce/train_tfidf.csv')
train_enc = pd.read_csv('../MachineHackE-Commerce/train_enc.csv')

In [4]:
test_embedding = pd.read_csv('../MachineHackE-Commerce/test_embedding.csv')
test_topic = pd.read_csv('../MachineHackE-Commerce/test_topic.csv')
test_tfidf = pd.read_csv('../MachineHackE-Commerce/test_tfidf.csv')
test_enc = pd.read_csv('../MachineHackE-Commerce/test_enc.csv')

In [5]:
train_enc1 = train_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2','Selling_Price'],axis=1)*1
test_enc1 = test_enc.drop(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2'],axis=1)*1

In [6]:
def f_y(s):
    return s/1000

In [7]:
temptrain = f_y(train_enc1['Year'])
temptest = f_y(test_enc1['Year'])

In [8]:
train_enc1['Year'] = temptrain
test_enc1['Year'] = temptest

In [9]:
def f(s):
    return s/s.max()

In [10]:
train_enc1['Month'] = train_enc1[['Month']].apply(f)
test_enc1['Month'] = test_enc1[['Month']].apply(f)


train_enc1['Week'] = train_enc1[['Week']].apply(f)
test_enc1['Week'] = test_enc1[['Week']].apply(f)

train_enc1['Day'] = train_enc1[['Day']].apply(f)
test_enc1['Day'] = test_enc1[['Day']].apply(f)

train_enc1['Dayofweek'] = train_enc1[['Dayofweek']].apply(f)
test_enc1['Dayofweek'] = test_enc1[['Dayofweek']].apply(f)


In [11]:
encoded = pd.concat([train_enc1,test_enc1]).values
encoded.shape

(3503, 11)

In [12]:
train_embedding.shape
embedding = pd.concat([train_embedding,test_embedding]).values
embedding.shape

(3503, 768)

In [13]:
train_topic.shape
topic = pd.concat([train_topic,test_topic]).values
topic.shape

(3503, 15)

In [14]:
train_tfidf.shape
tfidf = pd.concat([train_tfidf,test_tfidf]).values
tfidf.shape

(3503, 100)

In [15]:
NUM_BRANDS = 790
NUM_CATEGORIES = 1001
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 3

In [16]:
nrow_train = train.shape[0]
y = np.log1p(train["Selling_Price"])
#y = train["Selling_Price"]
dataset = pd.concat([train, test])
      

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [17]:
dataset['Item_Category'] = dataset['Item_Category'].astype('category')
dataset['Product_Brand'] = dataset['Product_Brand'].astype('category')
dataset['Subcategory_1'] = dataset['Subcategory_1'].astype('category')
dataset['Subcategory_2'] = dataset['Subcategory_2'].astype('category')

dataset['text'] = dataset.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [18]:
cv = CountVectorizer(min_df=NAME_MIN_DF)
X_Item_Category = cv.fit_transform(dataset['Item_Category'])
    

In [19]:
cv = CountVectorizer()
X_category1 = cv.fit_transform(dataset['Subcategory_1'])
    

In [20]:
cv2 = CountVectorizer()
X_category2 = cv.fit_transform(dataset['Subcategory_2'])
    

In [21]:
tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
X_description = tv.fit_transform(dataset['text'])
    

In [22]:
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(dataset['Product_Brand'])
    

In [23]:
dataset['Product'] = dataset['Product'].str.replace(r'\D', '')

dataset['Product'] = pd.to_numeric(dataset['Product'], errors='coerce')


In [24]:
X_product = f(dataset['Product'])

In [25]:
X_prod=np.array(X_product).reshape(-1,1)

In [26]:
X_description.shape
X_category1.shape
X_category2.shape
X_Item_Category.shape
X_prod.shape

(3503, 1)

In [27]:
sparse_merge1 = hstack((X_description, X_brand, X_category1,X_category2, X_Item_Category , X_prod,encoded)).tocsr()
X1 = sparse_merge1[:nrow_train]
X_test1 = sparse_merge1[nrow_train:]
   

In [28]:
def rmsle(y, y_pred):
        return np.sqrt(mean_squared_log_error(y, y_pred))

In [29]:
model_ridge =Ridge(solver="sag", fit_intercept=True, random_state=2)

model_ridge.fit(X1, y)

model_ridge_train_pred = np.expm1(model_ridge.predict(X1))

model_ridge_pred = np.expm1(model_ridge.predict(X_test1))

print(rmsle(np.expm1(y), model_ridge_train_pred))

0.38174047029808633


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


In [30]:
model_ridge_pred

array([2042.09116064,  527.75023581,  637.09489965, ..., 1349.56415152,
        386.44812211, 1467.3775975 ])

In [31]:
residue = train.Selling_Price - model_ridge_train_pred

In [32]:
residue

0      -237.090701
1       316.430075
2       194.418526
3       -23.553862
4       -87.364022
           ...    
2447    -23.705270
2448    570.767693
2449    226.849395
2450    242.325790
2451   -285.451002
Name: Selling_Price, Length: 2452, dtype: float64

In [33]:
res = Ridge(solver="sag", fit_intercept=True, random_state=2)

res.fit(X1,residue)

train_pred_res = res.predict(X1)

pred_res = res.predict(X_test1)



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


In [35]:
train_pred_res

array([337.36768715, -99.82867153, 124.18031687, ..., 104.8026609 ,
        47.00135655, 137.56350681])

In [36]:
residue

0      -237.090701
1       316.430075
2       194.418526
3       -23.553862
4       -87.364022
           ...    
2447    -23.705270
2448    570.767693
2449    226.849395
2450    242.325790
2451   -285.451002
Name: Selling_Price, Length: 2452, dtype: float64

In [38]:
model_ridge_pred + pred_res

array([2127.37956371, 2503.0446109 ,  526.56426268, ..., 1416.96615086,
        344.57956191, 1691.03907122])

In [ ]:
sub['Selling_Price'] = ensemble

sub.to_excel("submission_ridge_rf_X3.xlsx", index=False)
